<a href="https://colab.research.google.com/github/chrisporras/graddiss/blob/main/GradientDissentersHackathon2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hackathon 2023
ML for Biomedical Data Science
Team: Gradient Dissenters
Members: Audrey Lee, Christian Porras, Joy Jiang

## Install and load Kaggle mammography

In [ ]:
# Clone project git repo
!git clone https://github.com/chrisporras/graddiss.git

fatal: destination path 'graddiss' already exists and is not an empty directory.


In [ ]:
# install Kaggle public api
! pip install -q kaggle
# Choose the kaggle.json file that you downloaded
! mkdir ~/.kaggle
! cp ./graddiss/kaggle.json ~/.kaggle/
# Make directory named kaggle and copy kaggle.json file there.
!chmod 600 ~/.kaggle/kaggle.json
#Change the permissions of the file.
! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                      title                                     size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ---------------------------------------  -----  -------------------  -------------  ---------  ---------------  
themrityunjaypathak/covid-cases-and-deaths-worldwide     Covid Cases and Deaths WorldWide           8KB  2023-02-01 12:22:51          11378        379  1.0              
naveenkumar20bps1137/walmart-sales-analysis              WALMART SALES ANALYSIS                   119KB  2023-03-03 18:02:11           1811         49  0.9411765        
datascientistanna/customers-dataset                      Shop Customer Data                        23KB  2023-02-07 18:42:21           9799        222  1.0              
ulrikthygepedersen/co2-emissions-by-country              CO2 Emissions                    

In [ ]:
!kaggle competitions download -c mammography-image-patch-classification-2023

 97% 446M/460M [00:03<00:00, 163MB/s]
100% 460M/460M [00:03<00:00, 149MB/s]


In [ ]:
# data directory
!mkdir data
# unzip data there,
!unzip mammography-image-patch-classification-2023.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/train_imgs/P000435.png  
  inflating: data/train_imgs/P000436.png  
  inflating: data/train_imgs/P000437.png  
  inflating: data/train_imgs/P000438.png  
  inflating: data/train_imgs/P000439.png  
  inflating: data/train_imgs/P000440.png  
  inflating: data/train_imgs/P000441.png  
  inflating: data/train_imgs/P000442.png  
  inflating: data/train_imgs/P000443.png  
  inflating: data/train_imgs/P000444.png  
  inflating: data/train_imgs/P000445.png  
  inflating: data/train_imgs/P000446.png  
  inflating: data/train_imgs/P000447.png  
  inflating: data/train_imgs/P000448.png  
  inflating: data/train_imgs/P000449.png  
  inflating: data/train_imgs/P000450.png  
  inflating: data/train_imgs/P000451.png  
  inflating: data/train_imgs/P000452.png  
  inflating: data/train_imgs/P000453.png  
  inflating: data/train_imgs/P000454.png  
  inflating: data/train_imgs/P000455.png  
  inflating: data/train_imgs/P000456.png  
  i

## Prepare data by creating datasets and loaders.

In [ ]:
# Configuration class
class Config:
    resize = True

In [ ]:
# CustomImageDataset class
import numpy as np
import os
import torch
from PIL import Image
import torchvision
from skimage import io, img_as_float32
import numpy as np
import torch

class ToTensor3D(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, image):
        image = img_as_float32(image)
        # The following two lines are to duplicate the grayscale image onto
        # 3 channels. That's to make the input compatible with those models
        # trained on color images. If you don't use pretrained models, you can
        # delete these two lines.
        new_shape = (3,) + image.shape
        dup_img = np.broadcast_to(image, new_shape)
        return torch.from_numpy(dup_img.copy())

transform = ToTensor3D()

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, df):
        # Create one iterable that get be __getitemed__
        self.image_dir = image_dir
        self.df = df

    def __len__(self): # Denotes the total number of samples
        return self.df.shape[0]

    def __getitem__(self, index): # Generates one sample of data
        label = torch.tensor(self.df.iloc[index][['label']],dtype=torch.long) 

        # Goes to image_tensor below
        image_path = os.path.join(self.image_dir, self.df.iloc[index]['img_name'])
        image_og = Image.open(image_path)
        image = transform(image_og)

        # Transforms: Efficientnet B4: 380
        if Config.resize:
            resize = torchvision.transforms.Resize(384)
            image = resize(image)

        print("All done.")
        
        return image, label

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd

df = pd.read_csv('/sc/arion/projects/EHR_ML/joy/Hackathon2023/train.csv') 
train_dataset = CustomImageDataset('/sc/arion/projects/EHR_ML/joy/Hackathon2023/train_imgs', df)

print(train_dataset)